In [1]:
import pyactr as actr

robot_mind   = actr.ACTRModel()


In [3]:
for i in dir(robot_mind):
    print(i)

MODEL_PARAMETERS
_ACTRModel__buffers
_ACTRModel__env
_ACTRModel__similarities
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
chunkstring
chunktype
decmem
decmems
goal
goals
model_parameters
productions
productionstring
retrieval
retrievals
set_decmem
set_goal
set_productions
set_retrieval
set_similarities
simulation
visbuffers
visualBuffer


In [2]:
dm_module=robot_mind.decmem 
goal_buffer=robot_mind.goal
rtrv_buffer=robot_mind.retrieval


# Assumption - Robot knows the world and it is stored in declarative memory
- We consider world to be 3 x 15 grid,
- Robot will start moving in the second column, whenever there is an obstacle the robot will push the obstacle to either side.
- if obstacle slide is not possible, it will pick up and place it in behind cell.
- lets call only second column of the world as cells. 
## chunk definitions



In [3]:
actr.chunktype("world_cell", "name, left, right, block, next")# to be strored in declarative memory
actr.chunktype("goal_chunk", "start, end, current_pos, nxt_pos,status , state")
actr.chunktype("intrnl_rep","object, left, right")

## make world chunks

In [4]:
cell1 = actr.makechunk(nameofchunk='w1',typename='world_cell',
                       name=1,block=False,left=False,right=False,next=2)
cell2 = actr.makechunk(nameofchunk='w2',typename='world_cell',
                       name=2,block=True,left=False,right=False,next=3)
cell3 = actr.makechunk(nameofchunk='w3',typename='world_cell',
                       name=3,block=True,left=True,right=False,next=4)
cell4 = actr.makechunk(nameofchunk='w4',typename='world_cell',
                       name=4,block=True,left=True,right=True,next=5)
cell5 = actr.makechunk(nameofchunk='w5',typename='world_cell',
                       name=5,block=True,left=True,right=True,next=None)

In [5]:
dm_module.add(cell1)
dm_module.add(cell2)
dm_module.add(cell3)
dm_module.add(cell4)
dm_module.add(cell5)


In [6]:
dm_module

{world_cell(block= False, left= False, name= 1, next= 2, right= False): array([0.]), world_cell(block= True, left= False, name= 2, next= 3, right= False): array([0.]), world_cell(block= True, left= True, name= 3, next= 4, right= False): array([0.]), world_cell(block= True, left= True, name= 4, next= 5, right= True): array([0.]), world_cell(block= True, left= True, name= 5, next= None, right= True): array([0.])}

# creating goal buffer

In [7]:
#creating goal buffer
goal_buffer.add(actr.chunkstring(string="""
    isa     goal_chunk
    start   1
    end     5
    current_pos 1
    nxt_pos     2
    state INITIAL
    status FREE

"""))

In [8]:
goal_buffer

{goal_chunk(current_pos= 1, end= 5, nxt_pos= 2, start= 1, state= INITIAL, status= FREE)}

# production rules follow; using productionstring, 

In [9]:
### Initialise
robot_mind.productionstring(name="Start", string="""
    =g>
    isa     goal_chunk
    status  FREE
    ==>
    =g>
    isa     goal_chunk
    status STOP                     
    +manual>
    isa _manual
    cmd ROS_stop
    
""")

####stop decision
robot_mind.productionstring(name="stop", string="""
    =g>
    isa     goal_chunk
    status STOP
    ==>
    =g>
    isa     goal_chunk
    status FREE                     
    +manual>
    isa _manual
    cmd ROS_move
""")
###show current position
robot_mind.productionstring(name='show',string="""
    =g>
    isa goal_chunk
    state SHOW
    ==>
    !g>
    show current_pos
    =g>
    isa goal_chunk
    state MAKE_DECISION
                            
                            """)

###move to cell, this operation directly fetches next cell data ater moving
robot_mind.productionstring(name="move", string="""
    =g>
    isa     goal_chunk
    state   MOVE    
    current_pos   =x
    end ~=x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    ==>
    
    =g>
    isa     goal_chunk
    current_pos   =y
    nxt_pos       =z
    state SHOW
    
    +retrieval>
    isa world_cell
    name   =z
""")

### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_no_obstacle", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block False 
                    
    ==>
    =g>
    isa     goal_chunk
    state MOVE
""")

### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_obstacle_lf_rf", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block True
    left  False
    right False

                    
    ==>
    !g>
    show 'push block left @' 
    show nxt_pos 
    =g>
    isa     goal_chunk
    state MOVE
""")

### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_obstacle_l_or_rf", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block True
    left  =flag
    right ~=flag

                    
    ==>
    !g>
    show 'push block to free side @' 
    show nxt_pos 
    =g>
    isa     goal_chunk
    state MOVE
""")
### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_pickup", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block True
    left  True
    right True

                    
    ==>
    !g>
    show 'pick obstacle @' 
    show nxt_pos 
    =g>
    isa     goal_chunk
    state MOVE
    status LOAD
    
""")


### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_release", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status  LOAD
    current_pos  =x
    nxt_pos       =y
                        
    ==>
    !g>
    show 'Turn around @' 
    show current_pos
    show 'release load, and Turn around @'
    show current_pos
    =g>
    isa     goal_chunk
    status FREE
""")



{'=g': goal_chunk(current_pos= =x, end= , nxt_pos= =y, start= , state= MAKE_DECISION, status= LOAD)}
==>
{'!g': ParseResults([ParseResults(['show', "'Turn around @'"], {}), ParseResults(['show', 'current_pos'], {}), ParseResults(['show', "'release load, and Turn around @'"], {}), ParseResults(['show', 'current_pos'], {})], {}), '=g': goal_chunk(current_pos= , end= , nxt_pos= , start= , state= , status= FREE)}

In [10]:
#creating goal buffer
goal_buffer.add(actr.chunkstring(string="""
    isa     goal_chunk    
    status FREE

"""))
robot_sim=robot_mind.simulation()
robot_sim.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: Start')
(0.05, 'PROCEDURAL', 'RULE FIRED: Start')
(0.05, 'g', 'MODIFIED')
(0.05, 'manual', 'COMMAND: ROS_stop')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: stop')
(0.1, 'manual', 'ROS REQUEST SENT')
ROS_stop
(0.1, 'PROCEDURAL', 'RULE FIRED: stop')
(0.1, 'g', 'MODIFIED')
(0.1, 'manual', 'COMMAND: ROS_move')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'RULE SELECTED: Start')
(0.15, 'manual', 'ROS REQUEST SENT')
ROS_move
(0.15, 'PROCEDURAL', 'RULE FIRED: Start')
(0.15, 'g', 'MODIFIED')
(0.15, 'manual', 'COMMAND: ROS_stop')
(0.15, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.15, 'PROCEDURAL', 'RULE SELECTED: stop')
(0.2, 'manual', 'ROS REQUEST SENT')
ROS_stop
(0.2, 'PROCEDURAL', 'RULE FIRED: stop')
(0.2, 'g', 'MODIFIED')
(0.2, 'manual', 'COMMAND: ROS_move')
(0.2, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.2, 'PROCEDURAL', 'RULE SELECTED: Start')
(0.25, 'manual', 'ROS R

/home/bhanu/miniconda3/envs/robotics/lib/python3.11/site-packages/pyactr-0.3.1-py3.11.egg/pyactr/simulation.py:169: UserWarning: Process in manual interrupted
  warnings.warn("Process in %s interrupted" % name)


In [11]:
{'=g': goal_chunk(current_pos= =x, end= , nxt_pos= =y, start= , state= MAKE_DECISION, status= LOAD)}
==>
{'!g': ParseResults([ParseResults(['show', "'Turn around @'"], {}), ParseResults(['show', 'current_pos'], {}), ParseResults(['show', "'release load, and Turn around @'"], {}), ParseResults(['show', 'current_pos'], {})], {}), '=g': goal_chunk(current_pos= , end= , nxt_pos= , start= , state= , status= FREE)}
(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: Start')
(0.05, 'PROCEDURAL', 'RULE FIRED: Start')
(0.05, 'g', 'MODIFIED')
(0.05, 'manual', 'COMMAND: ROS_move')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: stop')
(0.1, 'manual', 'ROS REQUEST SENT')
(0.1, 'PROCEDURAL', 'RULE FIRED: stop')
done
(0.1, 'g', 'EXECUTED')
(0.1, 'g', 'CLEARED')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'NO RULE FOUND')
(0.25, 'manual', 'RESPONSE FROM ROS RECIEVED')
(0.25, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.25, 'PROCEDURAL', 'NO RULE FOUND')

SyntaxError: invalid syntax (2669917559.py, line 1)

In [ ]:
import socket

In [ ]:
message='c'
sender_socket.send(message.encode())